### 01. 그룹화 및 Aggregation

```df.groupby(그룹핑할 기준)[그룹핑할 데이터]``` + ```sum```/```mean```/```count``` 등

어떤 키를 기준으로 데이터를 묶은 뒤 각 그룹에 대한 통계치 계산

In [1]:
# import seaborn as sns

# tips = sns.load_dataset('tips')

import pandas as pd
tips = pd.read_csv('../data/tips.csv')
tips

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4
...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,Dinner,3
240,27.18,2.00,Female,Yes,Sat,Dinner,2
241,22.67,2.00,Male,Yes,Sat,Dinner,2
242,17.82,1.75,Male,No,Sat,Dinner,2


In [3]:
tips.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   total_bill  244 non-null    float64
 1   tip         244 non-null    float64
 2   sex         244 non-null    object 
 3   smoker      244 non-null    object 
 4   day         244 non-null    object 
 5   time        244 non-null    object 
 6   size        244 non-null    int64  
dtypes: float64(2), int64(1), object(4)
memory usage: 13.5+ KB


In [ ]:
# 요일(day)별 팁(tip)의 평균
tips.groupby('day')['tip'].mean()

day
Fri     2.734737
Sat     2.993103
Sun     3.255132
Thur    2.771452
Name: tip, dtype: float64

In [ ]:
# 흡연 여부별 total_bill 평균
tips.groupby('smoker')['total_bill'].mean()

smoker
No     19.188278
Yes    20.756344
Name: total_bill, dtype: float64

In [ ]:
# 시간별 요일의 합(?)
tips.groupby('time')['day'].sum() # --> 문자는 집계함수가 통하지 않는다!

time
Dinner    SunSunSunSunSunSunSunSunSunSunSunSunSunSunSunS...
Lunch     ThurThurThurThurThurThurThurThurThurThurThurTh...
Name: day, dtype: object

```agg()``` : 각 그룹에 대해 여러 가지 통계를 한 번에 보고 싶을 때 사용

In [ ]:
# 요일별 팀의 평균과 합
tips.groupby('day')['tip'].agg(['mean', 'sum'])

# 왜 금요일에 팁이 적지?

,mean,sum
day,,
Fri,2.734737,51.96
Sat,2.993103,260.40
Sun,3.255132,247.39
Thur,2.771452,171.83


In [7]:
# 요일별(day) 그리고 흡연여부별(smoker) 그룹을 나눈 뒤 행 개수(식사건수) 구하라
result = tips.groupby(['day', 'smoker'])['tip'].count()
result

day   smoker
Fri   No         4
      Yes       15
Sat   No        45
      Yes       42
Sun   No        57
      Yes       19
Thur  No        45
      Yes       17
Name: tip, dtype: int64

`reset_index()` : 기존 데이터의 인덱스들을 컬럼으로 넣고 일반적인 인덱스로 초기화
- 괄호 안에 `name = '~~~'` 를 작성해주면, 기존 인덱스의 컬럼명이 됨.
- 괄호를 비워두면 기존 속성명이 기본값

In [8]:
result.reset_index(name='카운트')

,day,smoker,카운트
0,Fri,No,4
1,Fri,Yes,15
2,Sat,No,45
3,Sat,Yes,42
4,Sun,No,57
5,Sun,Yes,19
6,Thur,No,45
7,Thur,Yes,17


```agg()```에 딕셔너리를 사용하면 컬럼마다 다른 집계 함수 적용 가능

In [17]:
# 요일별로 total_bill과 tip의 합계와 평균을 구하라
result2 = tips.groupby('day').agg({
    'total_bill': 'sum',    # total_bill은 합계로
    'tip': 'mean'           # tip은 평균으로
})

result2 

,total_bill,tip
day,,
Fri,325.88,2.734737
Sat,1778.40,2.993103
Sun,1627.16,3.255132
Thur,1096.33,2.771452


In [23]:
result2.reset_index()

,day,total_bill,tip
0,Fri,325.88,2.734737
1,Sat,1778.40,2.993103
2,Sun,1627.16,3.255132
3,Thur,1096.33,2.771452


### 02. 문자열 처리

- `Series.str.lower()` : 모두 소문자 변환
  
- `Series.str.upper()` : 대문자
  
- `Series.str.title()` : 단어마다 첫 글자만 대문자
  
- `Series.str.strip()`: 좌우 공백 제거 (특정 문자 제거는 `.strip('x')`)
  
- `Series.str.contains('문자열')`: 각 문자열이 특정 패턴을 포함하는지 True/False
  
- `Series.str.replace('패턴', '대체')`: 문자열 치환 (정규식 패턴 사용 가능, 정규식 아닌 단순 치환은 `regex=False` 지정)
  
- `Series.str.split('구분자')`: 구분자로 분할 -> 리스트 반환 (이후 `.str[0]`으로 첫 부분 추출 등 가능)
  
- `Series.str.cat(sep=',')`: 문자열 연결 (리스트처럼 이어붙이기)
  
- `Series.str.len()`: 문자열 길이 반환 (숫자에는 NaN)

이 외에도 부분 문자열 추출 (`slice`), 정규표현식 `extract`, `startswith`, `endswith`, `find` 등등 다양한 메서드가 있습니다.

In [24]:
products = pd.Series([
    "Bush Somerset Collection Bookcase",
    "Hon Deluxe Fabric Upholstered Stacking Chairs, Rounded Back",
    "Self-Adhesive Address Labels",
    "Staples 8.5x11 Copy Paper"
])

In [27]:
products.str.upper()

0                    BUSH SOMERSET COLLECTION BOOKCASE
1    HON DELUXE FABRIC UPHOLSTERED STACKING CHAIRS,...
2                         SELF-ADHESIVE ADDRESS LABELS
3                            STAPLES 8.5X11 COPY PAPER
dtype: object

In [29]:
products.str.strip()

0                    Bush Somerset Collection Bookcase
1    Hon Deluxe Fabric Upholstered Stacking Chairs,...
2                         Self-Adhesive Address Labels
3                            Staples 8.5x11 Copy Paper
dtype: object

In [26]:
#상품명에 "Bookcase"라는 단어가 들어가는지를 찾아라
contains_bookcase = products.str.contains("Bookcase")
contains_bookcase

# 첫 번째 제품명에만 포함되어 있으므로 True, 나머지는 False

0     True
1    False
2    False
3    False
dtype: bool

In [31]:
products.str.replace('a', '@')

0                    Bush Somerset Collection Bookc@se
1    Hon Deluxe F@bric Upholstered St@cking Ch@irs,...
2                         Self-Adhesive Address L@bels
3                            St@ples 8.5x11 Copy P@per
dtype: object

In [32]:
underscored = products.str.replace(" ", "_", regex=False)
print(underscored)

0                    Bush_Somerset_Collection_Bookcase
1    Hon_Deluxe_Fabric_Upholstered_Stacking_Chairs,...
2                         Self-Adhesive_Address_Labels
3                            Staples_8.5x11_Copy_Paper
dtype: object


### 03. 시간 데이터 처리

#### 날짜 형식 변환 (`pd.to_datetime`) : 문자열을 날짜/시간으로 파싱

In [34]:
orders = pd.read_csv('../data/superstore_orders.csv')
orders.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales
0,1,CA-2017-152156,08/11/2017,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600
1,2,CA-2017-152156,08/11/2017,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400
2,3,CA-2017-138688,12/06/2017,16/06/2017,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036.0,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200
3,4,US-2016-108966,11/10/2016,18/10/2016,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775
4,5,US-2016-108966,11/10/2016,18/10/2016,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680


In [37]:
orders.info()
# 날짜형식인데 데이터타입은 object임. --> 형변환 해야겠다.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9800 entries, 0 to 9799
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Row ID         9800 non-null   int64  
 1   Order ID       9800 non-null   object 
 2   Order Date     9800 non-null   object 
 3   Ship Date      9800 non-null   object 
 4   Ship Mode      9800 non-null   object 
 5   Customer ID    9800 non-null   object 
 6   Customer Name  9800 non-null   object 
 7   Segment        9800 non-null   object 
 8   Country        9800 non-null   object 
 9   City           9800 non-null   object 
 10  State          9800 non-null   object 
 11  Postal Code    9789 non-null   float64
 12  Region         9800 non-null   object 
 13  Product ID     9800 non-null   object 
 14  Category       9800 non-null   object 
 15  Sub-Category   9800 non-null   object 
 16  Product Name   9800 non-null   object 
 17  Sales          9800 non-null   float64
dtypes: float

In [41]:
orders['Order Date'] = pd.to_datetime(orders['Order Date'], format='%d/%m/%Y')

In [42]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9800 entries, 0 to 9799
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Row ID         9800 non-null   int64         
 1   Order ID       9800 non-null   object        
 2   Order Date     9800 non-null   datetime64[ns]
 3   Ship Date      9800 non-null   object        
 4   Ship Mode      9800 non-null   object        
 5   Customer ID    9800 non-null   object        
 6   Customer Name  9800 non-null   object        
 7   Segment        9800 non-null   object        
 8   Country        9800 non-null   object        
 9   City           9800 non-null   object        
 10  State          9800 non-null   object        
 11  Postal Code    9789 non-null   float64       
 12  Region         9800 non-null   object        
 13  Product ID     9800 non-null   object        
 14  Category       9800 non-null   object        
 15  Sub-Category   9800 n

In [47]:
orders = orders.set_index('Order Date') # 인덱스처리
orders.head()

,level_0,index,Row ID,Order ID,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales
Order Date,,,,,,,,,,,,,,,,,,,
2017-11-08,0,0,1,CA-2017-152156,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600
2017-11-08,1,1,2,CA-2017-152156,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400
2017-06-12,2,2,3,CA-2017-138688,16/06/2017,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036.0,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200
2016-10-11,3,3,4,US-2016-108966,18/10/2016,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775
2016-10-11,4,4,5,US-2016-108966,18/10/2016,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680


In [48]:
orders.index

DatetimeIndex(['2017-11-08', '2017-11-08', '2017-06-12', '2016-10-11',
               '2016-10-11', '2015-06-09', '2015-06-09', '2015-06-09',
               '2015-06-09', '2015-06-09',
               ...
               '2018-03-27', '2015-05-21', '2015-05-21', '2015-05-21',
               '2015-05-21', '2017-05-21', '2016-01-12', '2016-01-12',
               '2016-01-12', '2016-01-12'],
              dtype='datetime64[ns]', name='Order Date', length=9800, freq=None)

In [51]:
orders = pd.read_csv('../data/superstore_orders.csv')
dates = pd.to_datetime(orders['Order Date'], format='%d/%m/%Y')
print(dates.head())

0   2017-11-08
1   2017-11-08
2   2017-06-12
3   2016-10-11
4   2016-10-11
Name: Order Date, dtype: datetime64[ns]


#### 날짜/시간 속성 접근 (dt 접근자)

- `Series.dt.year`, `month`, `day`, `hour`, `minute` 등: 연, 월, 일, 시, 분...
- `Series.dt.day_name()`: 요일 이름 (예: Monday)
- `Series.dt.weekday`: 요일 번호 (월=0,...일=6)
- `Series.dt.quarter`: 분기(1~4)
- `Series.dt.days_in_month`: 그 달의 일 수 등.

In [56]:
dates.dt.year
dates.dt.month
dates.dt.day

0        8
1        8
2       12
3       11
4       11
        ..
9795    21
9796    12
9797    12
9798    12
9799    12
Name: Order Date, Length: 9800, dtype: int32

In [54]:
orders['level_0'] = dates.dt.year
orders

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,level_0
0,1,CA-2017-152156,08/11/2017,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2017
1,2,CA-2017-152156,08/11/2017,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,2017
2,3,CA-2017-138688,12/06/2017,16/06/2017,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036.0,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2017
3,4,US-2016-108966,11/10/2016,18/10/2016,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,2016
4,5,US-2016-108966,11/10/2016,18/10/2016,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9795,9796,CA-2017-125920,21/05/2017,28/05/2017,Standard Class,SH-19975,Sally Hughsby,Corporate,United States,Chicago,Illinois,60610.0,Central,OFF-BI-10003429,Office Supplies,Binders,"Cardinal HOLDit! Binder Insert Strips,Extra St...",3.7980,2017
9796,9797,CA-2016-128608,12/01/2016,17/01/2016,Standard Class,CS-12490,Cindy Schnelling,Corporate,United States,Toledo,Ohio,43615.0,East,OFF-AR-10001374,Office Supplies,Art,"BIC Brite Liner Highlighters, Chisel Tip",10.3680,2016
9797,9798,CA-2016-128608,12/01/2016,17/01/2016,Standard Class,CS-12490,Cindy Schnelling,Corporate,United States,Toledo,Ohio,43615.0,East,TEC-PH-10004977,Technology,Phones,GE 30524EE4,235.1880,2016
9798,9799,CA-2016-128608,12/01/2016,17/01/2016,Standard Class,CS-12490,Cindy Schnelling,Corporate,United States,Toledo,Ohio,43615.0,East,TEC-PH-10000912,Technology,Phones,Anker 24W Portable Micro USB Car Charger,26.3760,2016


### 04. 데이터 결합 기법 (`merge`, `join`, `concat` 등)

#### Merge vs Concat 요약
- **merge/join**: 두 표를 관계형으로 합치는 것 (공통 키 기반, 세로 방향으로 컬럼 확장).
- **concat**: 동일한 구조의 표들을 이어붙이는 것 (행 또는 열 방향 단순 연결).


#### 1) `Merge` (병합) / `Join` (조인)

- **merge/join**: 두 표를 관계형으로 합치는 것 (공통 키 기반, 세로 방향으로 컬럼 확장).
- **concat**: 동일한 구조의 표들을 이어붙이는 것 (행 또는 열 방향 단순 연결).

> 두 가지를 잘 구분해서 써야 합니다. 
>
> 예를 들어 고객명부와 주문내역을 합칠 땐 **merge**이고, 
>
> 1월 데이터와 2월 데이터를 합칠 땐 **concat**입니다.


In [58]:
orders = pd.DataFrame({
    'Order ID': [101, 102, 103],
    'Product': ['Bookcase', 'Chair', 'Lamp'],
    'Profit': [100, 50, 20]
})
returns = pd.DataFrame({
    'Order ID': [102, 105],
    'Returned': ['Yes', 'Yes']
})

In [59]:
orders

,Order ID,Product,Profit
0,101,Bookcase,100
1,102,Chair,50
2,103,Lamp,20


In [60]:
returns

,Order ID,Returned
0,102,Yes
1,105,Yes


In [62]:
# how=''를 적지 않아도 실행됨 <- 기본이 inner join
merged = orders.merge(returns, on='Order ID', how='left')
merged['Returned'] = merged['Returned'].fillna('No')
print(merged)

   Order ID   Product  Profit Returned
0       101  Bookcase     100       No
1       102     Chair      50      Yes
2       103      Lamp      20       No


#### 2) Concatenate (연결)

`pd.concat([df1, df2, ...], axis=0) ` : df를 행 방향으로 이어 붙임

- `axis = 0` : 행 기준으로 붙임
  
- `axis = 1` : 열 기준으로 붙임

- 인덱스가 겹칠 수 있으니 무시하려면 `ignore_index=True `

In [63]:
df1 = pd.DataFrame({'City': ['Seoul','Busan'], 'Sales':[100,150]})
df2 = pd.DataFrame({'City': ['Daegu','Incheon'], 'Sales':[80,70]})
combined = pd.concat([df1, df2], ignore_index=True)
print(combined)

      City  Sales
0    Seoul    100
1    Busan    150
2    Daegu     80
3  Incheon     70


### 05. apply, map, lambda 활용

#### apply 함수

- **`DataFrame.apply(func, axis=0)`** : 각 열(column)에 대해 함수를 적용 (default). axis=1로 지정하면 각 행(row)에 대해 함수를 적용합니다. 결과는 함수 반환에 따라 Series 또는 DataFrame이 될 수 있습니다.
  
- **`Series.apply(func)`**: Series의 각 값에 함수를 적용한 결과 Series를 반환합니다 (element-wise 적용).

df.apply(func, axis=0)

Series.apply(func)

In [65]:
tips

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4
...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,Dinner,3
240,27.18,2.00,Female,Yes,Sat,Dinner,2
241,22.67,2.00,Male,Yes,Sat,Dinner,2
242,17.82,1.75,Male,No,Sat,Dinner,2


In [71]:
# 방법 1
tips['tip_ratio'] = tips['tip'] / tips['total_bill']
tips

,total_bill,tip,sex,smoker,day,time,size,tip_ratio,tip_ratio2
0,16.99,1.01,Female,No,Sun,Dinner,2,0.059447,0.059447
1,10.34,1.66,Male,No,Sun,Dinner,3,0.160542,0.160542
2,21.01,3.50,Male,No,Sun,Dinner,3,0.166587,0.166587
3,23.68,3.31,Male,No,Sun,Dinner,2,0.139780,0.139780
4,24.59,3.61,Female,No,Sun,Dinner,4,0.146808,0.146808
...,...,...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,Dinner,3,0.203927,0.203927
240,27.18,2.00,Female,Yes,Sat,Dinner,2,0.073584,0.073584
241,22.67,2.00,Male,Yes,Sat,Dinner,2,0.088222,0.088222
242,17.82,1.75,Male,No,Sat,Dinner,2,0.098204,0.098204


In [72]:
# 방법 2

tips['tip_ratio2'] = tips.apply(lambda row : row['tip'] / row['total_bill'], axis = 1)
tips

,total_bill,tip,sex,smoker,day,time,size,tip_ratio,tip_ratio2
0,16.99,1.01,Female,No,Sun,Dinner,2,0.059447,0.059447
1,10.34,1.66,Male,No,Sun,Dinner,3,0.160542,0.160542
2,21.01,3.50,Male,No,Sun,Dinner,3,0.166587,0.166587
3,23.68,3.31,Male,No,Sun,Dinner,2,0.139780,0.139780
4,24.59,3.61,Female,No,Sun,Dinner,4,0.146808,0.146808
...,...,...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,Dinner,3,0.203927,0.203927
240,27.18,2.00,Female,Yes,Sat,Dinner,2,0.073584,0.073584
241,22.67,2.00,Male,Yes,Sat,Dinner,2,0.088222,0.088222
242,17.82,1.75,Male,No,Sat,Dinner,2,0.098204,0.098204


`Series.map(func or dict) `: Series의 각 요소를 주어진 함수에 매핑하여 새로운 Series를 반환

In [73]:
tips['gender_code'] = tips['sex'].map({'Male': 1, 'Female': 0})
print(tips[['sex','gender_code']].head(3))

      sex  gender_code
0  Female            0
1    Male            1
2    Male            1


In [74]:
tips

,total_bill,tip,sex,smoker,day,time,size,tip_ratio,tip_ratio2,gender_code
0,16.99,1.01,Female,No,Sun,Dinner,2,0.059447,0.059447,0
1,10.34,1.66,Male,No,Sun,Dinner,3,0.160542,0.160542,1
2,21.01,3.50,Male,No,Sun,Dinner,3,0.166587,0.166587,1
3,23.68,3.31,Male,No,Sun,Dinner,2,0.139780,0.139780,1
4,24.59,3.61,Female,No,Sun,Dinner,4,0.146808,0.146808,0
...,...,...,...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,Dinner,3,0.203927,0.203927,1
240,27.18,2.00,Female,Yes,Sat,Dinner,2,0.073584,0.073584,0
241,22.67,2.00,Male,Yes,Sat,Dinner,2,0.088222,0.088222,1
242,17.82,1.75,Male,No,Sat,Dinner,2,0.098204,0.098204,1


In [75]:
# 비흡연자 = 0 / 흡연자 = 1 -> 새로운 열 'smoker_code'

tips['smoker_code'] = tips['smoker'].map({'No':0, 'Yes':1})
tips

,total_bill,tip,sex,smoker,day,time,size,tip_ratio,tip_ratio2,gender_code,smoker_code
0,16.99,1.01,Female,No,Sun,Dinner,2,0.059447,0.059447,0,0
1,10.34,1.66,Male,No,Sun,Dinner,3,0.160542,0.160542,1,0
2,21.01,3.50,Male,No,Sun,Dinner,3,0.166587,0.166587,1,0
3,23.68,3.31,Male,No,Sun,Dinner,2,0.139780,0.139780,1,0
4,24.59,3.61,Female,No,Sun,Dinner,4,0.146808,0.146808,0,0
...,...,...,...,...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,Dinner,3,0.203927,0.203927,1,0
240,27.18,2.00,Female,Yes,Sat,Dinner,2,0.073584,0.073584,0,1
241,22.67,2.00,Male,Yes,Sat,Dinner,2,0.088222,0.088222,1,1
242,17.82,1.75,Male,No,Sat,Dinner,2,0.098204,0.098204,1,0


### 06. 전처리 완료 데이터 저장 (Saving Processed Data)

In [9]:
# df.to_csv('ames_processed.csv', index=False)
# print("전처리된 데이터를 ames_processed.csv 파일로 저장했습니다.")